<a href="https://colab.research.google.com/github/tyoc213/blog/blob/master/__explorations__/2020_31/BlurExampleTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200707" #"nightly"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version $VERSION > /dev/null
#import torch_xla.core.xla_model as xm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  34560      0 --:--:-- --:--:-- --:--:-- 34560
Copying gs://tpu-pytorch/wheels/torch-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
- [1 files][107.5 MiB/107.5 MiB]                                                
Operation completed over 1 objects/107.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][123.8 MiB/123.8 MiB]                                                
Operation completed over 1 objects/123.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.                                      
ERROR:

In [2]:
!pip install https://github.com/butchland/fastai_xla_extensions/archive/master.zip > /dev/null
import fastai_xla_extensions.core
#!pip install fastai2 > /dev/null



In [3]:
!pip install git+https://github.com/ohmeow/blurr > /dev/null

  Running command git clone -q https://github.com/ohmeow/blurr /tmp/pip-req-build-v7rka92k


# imports

In [4]:
import pyarrow
pyarrow.__version__, int(pyarrow.__version__.split(".")[1]) < 16, int(pyarrow.__version__.split(".")[0]) > 0
pyarrow.__version__ = "0.16.0"

In [5]:
import torch
from transformers import *
from fastai2.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

In [6]:
dede = default_device()
dede

device(type='xla', index=1)

# Data

In [7]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

# Get your hugs huggingface objects

In [8]:
task = HF_TASKS_AUTO.SequenceClassification

pretrained_model_name = "bert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,  task=task)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Build your bricksbricksbricks DataBlock bricksbricksbricks and your DataLoaders

In [9]:

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

dls = dblock.dataloaders(imdb_df, bs=4,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"i'm not sure it was the language or the poor acting, but everything about this movie feels and looks cheap and fake. < br / > < br / > after seeing der untergang this is a huge disappointment. there's no connection between different scenes, and the acting is so incredibly poor i couldn't even believe people could make such a mess of something that had great potential. < br / > < br / > and above all, everyone in germany speaks english. big mistake. the german language has a certain sound to it, and especially hitler himself only sounds like hitler when he's speaking / yelling german. < br / > < br / > the way the story is told made me believe it was improvised on the spot, the characters were empty and the movie seems to be a collection of random events that could have happened. < br / > < br / > whether it's the english or the fact that i've already seen der untergang, everything about this movie was fake and ridiculous.",negative


# ... and train steam_locomotive

In [10]:
%%time
#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.697442,0.712366,0.550000,00:56
1,0.619426,0.658342,0.580000,00:29
2,0.614199,0.602724,0.735000,00:29


/usr/local/lib/python3.6/dist-packages/fastai2/callback/schedule.py:68: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx = (pos >= pcts).nonzero().max()


CPU times: user 30.6 s, sys: 3.43 s, total: 34.1 s
Wall time: 2min


In [11]:
%%time
learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

,text,category,target
0,"this very funny british comedy shows what might happen if a section of london, in this case pimlico, were to declare itself independent from the rest of the uk and its laws, taxes & post - war restrictions. merry mayhem is what would happen. < br / > < br / > the explosion of a wartime bomb leads to the discovery of ancient documents which show that pimlico was ceded to the duchy of burgundy centuries ago, a small historical footnote long since forgotten. to the new burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from whitehall. < br / > < br / > stanley holloway is excellent as the minor city politician who suddenly finds himself leading one of the world's tiniest nations. dame margaret rutherford is a delight as the history professor who sides with pimlico. others in the stand - out cast include hermione baddeley, paul duplis, naughton wayne, basil radford & sir michael hordern. < br / > < br / > welcome to burgundy!",positive,positive
1,""" a cry in the dark "" is a masterful piece of cinema, haunting, and incredibly though provoking. the true story of lindy chamberland, who, in 1980, witnessed a horrific sight, seeing her 3 - month - old baby being brutally taken from their family's tent, while camping on the austrailian outback. azaria ( the baby ) was never seen again, and the result of her horrendous disappearance caused a true life frenzy all around the world. meryl streep does immaculate justice to the role of lindy, as she always does. but the one thing that helps "" a cry in the dark "" never fall flat is the brilliant direction. a truly inspired and accurate outlook on this baffeling case, tears are brought to the eyes. the concept is nothing less then terrifying, and afterwards you are left haunted, but also inspired.",positive,positive


CPU times: user 159 ms, sys: 14.6 ms, total: 174 ms
Wall time: 4.19 s


# Going wild with the batch size

In [13]:
%%time

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

dls = dblock.dataloaders(imdb_df, bs=4*4,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"if you liked the grinch movie... go watch that again, because this was no where near as good a seussian movie translation. mike myers'cat is probably the most annoying character to "" grace "" the screen in recent times. his voice / accent is terrible and he laughs at his own jokes with an awful weasing sound, which is about the only laughing i heard at the theater. not even the kids liked this one folks, and kids laugh at anything now. save your money and go see looney tunes : back in action if you're really looking for a fun holiday family movie.",negative


epoch,train_loss,valid_loss,accuracy,time
0,0.610127,0.593916,0.720000,00:58
1,0.596659,0.587926,0.720000,00:17
2,0.581165,0.596053,0.710000,00:16


,text,category,target
0,"this very funny british comedy shows what might happen if a section of london, in this case pimlico, were to declare itself independent from the rest of the uk and its laws, taxes & post - war restrictions. merry mayhem is what would happen. < br / > < br / > the explosion of a wartime bomb leads to the discovery of ancient documents which show that pimlico was ceded to the duchy of burgundy centuries ago, a small historical footnote long since forgotten. to the new burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from whitehall. < br / > < br / > stanley holloway is excellent as the minor city politician who suddenly finds himself leading one of the world's tiniest nations. dame margaret rutherford is a delight as the history professor who sides with pimlico. others in the stand - out cast include hermione baddeley, paul duplis, naughton wayne, basil radford & sir michael hordern. < br / > < br / > welcome to burgundy!",positive,positive
1,"with a title "" borrowed "" from werner herzog and liberal helpings of kubrick, haneke and noe it is painfully obvious that thomas clay considers himself a cut above the usual sort of rubbish our british cinema churns out. "" robert carmichael "" ( for short ) sets itself up as a realistic study of youthful alienation and at the same time seemingly a critique of the iraq war. the problem with the realism is that the characters are so patently unrealistic and atypical - contrary to the fetid imaginings of "" extreme "" filmmakers most teenagers are not drug addled rapists. as a critique of the iraq war, a film about youth violence ( by a talented classical musician - subtext society has damaged this sensitive individual ) is so infantile as to hardly bear thinking about. there are signs of technical ability but some reviewers have overstated this. like kubrick and noe he does show that the desire to shock linked with supposed serious intent may be the worst cinematic con trick of recent film. people liked "" clockwork orange "" and "" irreversible "" because they liked the rapes and the violence, but most of all they liked feeling culturally superior for liking things that most hated. so too much kubrick and not enough haneke ( a serious and moral filmmaker ) here labels this as one of the most moronic films in years. ( i am not against violence in film. to do it seriously is a hard trick though - people in cinemas cheered alex in "" clockwork orange "" showing how kubrick's supposed intent was missed by miles. gratuitous violence is much easier to achieve and is less offensive than the pretensions of many art - film directors. )",negative,negative


CPU times: user 14.9 s, sys: 1.25 s, total: 16.1 s
Wall time: 1min 43s


In [14]:
%%time

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

dls = dblock.dataloaders(imdb_df, bs=4*4*4,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"this movie raises a number of pressing questions in my mind. firstly, how has jennifer tilly managed to sustain a film acting career for all these years based on that ridiculous squeaky voice and the very limited range of hammy facial expressions she employs? secondly... what on earth were the people responsible for making this offensive and deeply repulsive film thinking of? and thirdly... given that there were people perverted enough to decide to make dreck like this, shouldn't there have been someone in the system - the studio, the distributors, or somewhere - sane enough to prevent it actually getting completed and released. you really would have to search a very, very long way to turn up another movie as profoundly nasty as this... and it isn't even billed as a horror movie - which, inasmuch as it can be seen as belonging to any legitimate film genre, it certainly is. the movie wallows from beginning to end in the sickest kind of madness, violence and abuse, and has essentially no redeeming features at all. i'm not actually advocating censorship ( which i don't believe in )... but i really can't see how anybody could conceivably draw anything positive from watching a film like this.",negative


epoch,train_loss,valid_loss,accuracy,time
0,0.590439,0.603719,0.690000,00:49
1,0.585435,0.573671,0.755000,00:13
2,0.575760,0.579840,0.730000,00:13


,text,category,target
0,"this very funny british comedy shows what might happen if a section of london, in this case pimlico, were to declare itself independent from the rest of the uk and its laws, taxes & post - war restrictions. merry mayhem is what would happen. < br / > < br / > the explosion of a wartime bomb leads to the discovery of ancient documents which show that pimlico was ceded to the duchy of burgundy centuries ago, a small historical footnote long since forgotten. to the new burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from whitehall. < br / > < br / > stanley holloway is excellent as the minor city politician who suddenly finds himself leading one of the world's tiniest nations. dame margaret rutherford is a delight as the history professor who sides with pimlico. others in the stand - out cast include hermione baddeley, paul duplis, naughton wayne, basil radford & sir michael hordern. < br / > < br / > welcome to burgundy!",positive,positive
1,"this movie had some andrenaline kickers, but it's an old story that simply could never happen. navy protocols could never break down that much that a crew much less an xo could ever go that far against the captain. i'll take dr. strangelove any day if i wish to see this plot. sidenote - - the us navy did not support this film.",negative,negative


CPU times: user 9.84 s, sys: 925 ms, total: 10.8 s
Wall time: 1min 31s


In [15]:
%%time

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

dls = dblock.dataloaders(imdb_df, bs=4*4*4*4,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"henri verneuil's film may be not so famous as parallax view, 3 days of the condor or jfk but it is certainly not worse and sometimes even better than these classic representatives of the genre. action takes place in fictional western state where fictional president has been killed. after several years of investigation, special government commission decides that president was killed by a lone gunman. but one man - prosecutor volney, played by yves montand - thinks there's something more to be investigated and so the film starts. this movie doesn't deal with some exact theories, but it embraces the whole structure of relationship between government and society in today's world. such film could be made only in the 1970 - ies but it will never lose it's actuality. furthermore, it's even a bit frightful how precise are it's oracles. 10 out of 10.",positive


RuntimeError: ignored

In [16]:
%%time

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

# half the resource exhaustion
dls = dblock.dataloaders(imdb_df, bs=4*4*4*2,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"oh dear. yet another example of "" oireland "" and religion. no doubt we'll be seeing some depressing nonsense featuring some "" hunky and macho freedom fighters "" from the ira. well that was my initial reaction when the credits started but just over an hour and a half later i was in a state of shock. what a superb movie < br / > < br / > the story starts on the day of the wedding between sean cloney and sheila kelly in the 1950s. there is a slight problem since they're getting married in the catholic church and that is sheila is a protestant but in order for the wedding to happen sheila takes a pledge that her children will be brought up catholic and attend the catholic school when they're old enough. the story - which is set in the 1950s - then jumps forward a few years when the cloney daughters are about to start school but sheila has decided they'll be attending the local protestant school much to the disgust of local priest father stafford. from there things escalate < br / > < br / > let me put my cards on the table and state that despite having both irish catholic and scottish protestant heritage i was brought up as agnostic and have considered myself as an atheist throughout my adult life. in fact when it comes to religion i consider myself a marxist and religion is a cynical weapon used to manipulate people. a love divided shows what happens when self appointed moral guardians take it upon themselves to tell other people what to think and believe. may i have the temerity to state that if karl marx saw this movie he'd love it and call it a masterpiece? perhaps i shouldn't since the drama of this story shows what happens when other people do your thinking for you < br / > < br / > in reply to the couple of reviewers who have claimed this movie is propaganda of the worst sort i don't claim to know the exact details of what happened in county wexford and there's no denying that father stafford and his flock of catholic sheep are portrayed as being the bad guys but sheila isn't blameless herself. think about a woman living in a rural village in 1950s ireland who takes a pledge to bring her children up as catholics then changes her mind and believes there will be no consequences of this? this is a warning against taking pledges and not keeping to them. not only that but she disappears to let other people pick up the pieces of their shattered lives. there's also something that no one else has picked up upon",positive


RuntimeError: ignored

In [18]:
%%time

# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))


# NOTE: I guess I have already exhausted memory
dls = dblock.dataloaders(imdb_df, bs=48,
                         device=dede
                         )
print(dls.device)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

xla:1


,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"i rated this film 7 / 10 which is an average of 8 / 10 for screenplay, direction and 1944 production values and 6 / 10 for acting. my acting rating in turn was calculated at 4 / 10 for all the screen characters except for that played by heroine ella raines as carol richman who was excellent at 8 / 10. also i commend thomas gomez as inspector burgess whose character convinces that he personally does not think the guilty verdict on scott henderson ( alan curtis ) was just in view of his naive alibi. these two then form an alliance to prove scott's alibi. < br / > < br / > i have this film on a "" suevia film noir cine negro "" dvd in spanish as "" la dama desconocida "" with the original soundtrack "" ingles "" as an alternative language, since despite searching i could not find a wholly english version. i was however anxious to see another performance by ella raines after being impressed with her performance as a heroine in "" impact "" playing a sole female garage proprietor. here ella performs another heroic role believing in the innocence of her engineer boss and refuses several suggestions that she should return to her home in kansas ( her boss's pet name for her ) before solving the missing alibi. the fact that she is secretly in love with her boss is a little hard to believe since he formally just seemed to have had a formal business relationship with her. he had however designed children's homes and playgrounds so i suppose "" family man "" had lit up in carol's brain. < br / > < br / > in the 1940s with "" the film code "" in operation, producers could only portray sex through metaphors and here it is done in the form of furious drumming played by elisha cooke jnr. carol dolls herself up as a girl of easy virtue in an attempt to lure the drummer into giving her information about "" the phantom lady "" alibi. the other main character, jack marlow ( an associate of scott henderson ) is played by franchot tone whose performance i found too theatrical and wondered why carol, for instance, did not notice him constantly and strangely admiring his hands. here the screenplay should have been improved and provided more suspense as these theatrical moves telegraphed the plot far too early to the audience.",positive


RuntimeError: ignored

In [19]:
4*4*4*4, 4*4*4*2

(256, 128)